In [5]:
# import important library
import pandas as pd
import numpy as np

In [8]:
# read the csv file
daily_df=pd.read_csv('BTC-Daily.csv')
daily_df.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
0,1646092800,2022-03-01 00:00:00,BTC/USD,43221.71,43626.49,43185.48,43185.48,49.006289,2.116360e+06
1,1646006400,2022-02-28 00:00:00,BTC/USD,37717.10,44256.08,37468.99,43178.98,3160.618070,1.364723e+08
2,1645920000,2022-02-27 00:00:00,BTC/USD,39146.66,39886.92,37015.74,37712.68,1701.817043,6.418008e+07
3,1645833600,2022-02-26 00:00:00,BTC/USD,39242.64,40330.99,38600.00,39146.66,912.724087,3.573010e+07
4,1645747200,2022-02-25 00:00:00,BTC/USD,38360.93,39727.97,38027.61,39231.64,2202.851827,8.642149e+07


In [ ]:
# Just for self-testing of the fitness function


In [ ]:
def bot_fitness_func(bot_signals, time):
    """
    This function will calculate the fitness (total cash earned from the buy/sell signals) of the trading bot
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time: recorded trading time (daily)
    Return:
    - cash: the total cash amount earned after the trading time
    """ 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0

    #loop through the time length
    for i in range(len(time)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
    
    return cash

In [ ]:
def bot_evaluation(bot_signals, time, close_price):
    """
    This function will calculate the total cash earned from the buy/sell signals of the trading bot.
    Parameters:
    - bot_signals (list): including the buy/sell and keep signal
    - time: recorded trading time (daily)
    Return:
    - result: a nested list, with each smallest contains the time of the transaction, the amount of cash and amount of bitcoin
    """ 
    # initial values
    cash = 1000
    fee=0.03
    bitcoin = 0.0
    result =[]

    #loop through the time length
    for i in range(len(time)-1):
        close_price=close[i]
        # buy
        if bot_signals[i] == "buy":
            bitcoin =  (cash*(1-fee))/close_price
            cash = 0
            result.append([time[i],cash, bitcoin])
        # sell
        elif bot_signals[i] == "sell":
            cash = bitcoin * close_price * (1-fee)
            bitcoin =0
            result.append([time[i],cash, bitcoin])
    
    # final evaluation to change back to cash
    close_price=close[-1]
    if bitcoin>0:
        cash = bitcoin * close_price * (1-fee)
        bitcoin =0
        result.append([time[i],cash, bitcoin])
    
    return result

In [ ]:
#def print_nicely(list):
    